# Generate all extra labels for efficient
0. img-width (pixel) (int)
1. 2d_box-Width (int)
2. 2d_box-Height (int)
3. offset-X (pixel) (int)
4. offset-Y (pixel) (int)
5. theta_ray (calib folder) (.3f)
6. group-alpha (+100*index)
7. group-rotation_y (+100*index)

In [ ]:
path = './Kitti/training'
extra_label_path = path + '/extra_label/'
idx = '000007'
extra_labels = open(extra_label_path + '%s.txt'%idx).read().splitlines()
print(extra_labels)
line_num = 2
print(int(extra_labels[line_num].split()[6])) # group_label

# FASTER Get Pred Labels (40 min->10)

In [2]:
from torchvision.models import vgg
from torch_lib.Model import *
from torch_lib.ClassAverages import *
from torchvision import transforms
import os, glob, cv2
from library.ron_utils import *

def get_calibration_cam_to_image(cab_f):
    for line in open(cab_f):
        if 'P2:' in line:
            cam_to_img = line.strip().split(' ')
            cam_to_img = np.asarray([float(number) for number in cam_to_img[1:]])
            cam_to_img = np.reshape(cam_to_img, (3, 4))
            return cam_to_img

def set_angle_bins(bins=2):
    angle_bins = torch.zeros(bins)
    interval = 2 * torch.pi / bins
    for i in range(1,bins):
        angle_bins[i] = i * interval
    angle_bins += interval / 2 # center of the bin
    return angle_bins

def Run_GT_pred_labels(weights_path, pred_label_root, cuda=0):
    device = torch.device(f'cuda:{cuda}')
    os.makedirs(pred_label_root, exist_ok=True)
    my_vgg = vgg.vgg19_bn(pretrained=True)
    model = Model(features=my_vgg.features, bins=2).to(device)
    
    #因為train的時候在不同GPU上，eval要map到同個GPU https://www.jianshu.com/p/ec91b3b59f66
    checkpoint = torch.load(weights_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    # for img processing
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    process = transforms.Compose([transforms.ToTensor(), normalize])

    # Kitti image_2 dir / label_2 dir
    img_root = "./Kitti/training/image_2"
    label_root = "./Kitti/training/label_2"
    calib_root = "./Kitti/training/calib"
    extra_label_root = "./Kitti/training/extra_label"

    images = glob.glob(os.path.join(img_root, '*.png'), recursive=True)
    labels = glob.glob(os.path.join(label_root, '*.txt'), recursive=True)
    calibs = glob.glob(os.path.join(calib_root, '*.txt'), recursive=True)
    extra = glob.glob(os.path.join(extra_label_root, '*.txt'), recursive=True)

    # dim averages
    averages_all = ClassAverages(classes=[], average_file='org_class_AVG.txt')
    angle_bins = set_angle_bins()
    start = time.time()
    for i in range(len(images)):
        img = cv2.imread(images[i])
        cam_to_img = get_calibration_cam_to_image(calibs[i])

        CLASSes = list()
        BOX2Ds = list()
        CROPs_tensor = list()
        THETAs = list()
        extra_labels = open(extra[i]).read().splitlines()


        with open(labels[i]) as f:
            lines = f.readlines()

            for idx, line in enumerate(lines):
                elements = line[:-1].split()
                if elements[0] == 'DontCare':
                    continue
                for j in range(1, len(elements)):
                    elements[j] = float(elements[j])

                CLASSes.append(elements[0])
                top_left = (int(round(elements[4])), int(round(elements[5])))
                btm_right = (int(round(elements[6])), int(round(elements[7])))
                box = [top_left, btm_right]
                BOX2Ds.append(box)
                #cv2 is(H,W,3)
                crop = img[top_left[1]:btm_right[1]+1, top_left[0]:btm_right[0]+1] 
                crop = cv2.resize(src = crop, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
                CROPs_tensor.append(process(crop))
                # Use calc function if cam_to_img changes to "camera_cal/calib_cam_to_cam.txt"
                #theta_ray =  calc_theta_ray(img.shape[1], box, cam_to_img)
                theta_ray = float(extra_labels[idx].split()[5])
                THETAs.append(theta_ray)

            # put together as a batch
            # model regress part
            input_ = torch.stack(CROPs_tensor).to(device)

            [ORIENTs, CONFs, delta_DIMs] = model(input_)
            
            max_indexes = torch.max(CONFs, dim=1)[1]
            orient_batch = ORIENTs[torch.arange(len(ORIENTs)), max_indexes]
            Alphas = torch.arctan2(orient_batch[:,1], orient_batch[:,0]) # arctan2(sin, cos)
            Alphas += angle_bins[max_indexes].to(device)
            Alphas -= torch.pi

        #write pred_label.txt 
        with open(labels[i].replace(label_root, pred_label_root),'w') as new_f:
            pred_labels = ''
            for class_, delta, alpha, theta, box_2d in zip(CLASSes, delta_DIMs, Alphas, THETAs, BOX2Ds):
                delta = delta.cpu().data.numpy() #torch->numpy
                alpha = alpha.cpu().data.numpy() #torch->numpy
                dim = delta + averages_all.get_item(class_)
                rotation_y = alpha + theta
                loc, _ = calc_location(dim, cam_to_img, box_2d, alpha, theta)

                pred_labels += '{CLASS} 0.0 0 {A:.2f} {left} {top} {right} {btm} {H:.2f} {W:.2f} {L:.2f} {X:.2f} {Y:.2f} {Z:.2f} {Ry:.2f}\n'.format(
                    CLASS=class_, A=alpha, left=box_2d[0][0], top=box_2d[0][1], right=box_2d[1][0], btm=box_2d[1][1],
                    H=dim[0], W=dim[1], L=dim[2], X=loc[0], Y=loc[1], Z=loc[2], Ry=rotation_y)
            #print(pred_labels)
            new_f.writelines(pred_labels)
        #print(pred_labels)
        if i%500==0:
            print(i)
    print('Done, take {} min {} sec'.format((time.time()-start)//60, (time.time()-start)%60))# around 10min

In [9]:
weights_path = 'weights_group/cuda0_w03_up10_100.pkl'
pred_label_root = 'cuda0_G_100'
Run_GT_pred_labels(weights_path, pred_label_root)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
Done, take 8.0 min 36.863417863845825 sec
